In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore')

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv("../data/8-1 train(std).csv",index_col=0)
test1 = pd.read_csv("../data/8-1 test1(std).csv",index_col=0)
test2 = pd.read_csv("../data/8-1 test2(std).csv",index_col=0)

print(train['label'].value_counts())
print(test1['label'].value_counts())
print(test2['label'].value_counts())

label
0    1223
1      88
Name: count, dtype: int64
label
0.0    3743
1.0      80
Name: count, dtype: int64
label
0.0    1911
1.0      98
Name: count, dtype: int64


데이터분할

In [3]:
x_train = train.drop("label",axis=1)
y_train = train["label"]
x_test =test1.drop("label",axis=1)
y_test = test1["label"]

오버샘플링 SMOTE

In [4]:
x_train, y_train = SMOTE(random_state=4).fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

(2446, 10)


label
0    1223
1    1223
Name: count, dtype: int64

In [5]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [6]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")



In [7]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [8]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()

lda_clf.fit(x_train, y_train)
lda_pred = lda_clf.predict(x_test)
lda_pred_proba = lda_clf.predict_proba(x_test)
print("---------------------------------------------------------") 
get_clf_eval(y_test, lda_pred)
get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)
    

---------------------------------------------------------
오차행렬
[[3212  531]
 [  24   56]]
정확도: 0.8548, 정밀도 : 0.0954, 재현율:0.7000,F1 스코어:0.1679
ROC 스코어: 0.7791, PR score : 0.0731
임곗값: 0.1
오차행렬
[[ 904 2839]
 [   6   74]]
정확도: 0.2558, 정밀도 : 0.0254, 재현율:0.9250,F1 스코어:0.0494
ROC 스코어: 0.5833, PR score : 0.0251
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[1766 1977]
 [   7   73]]
정확도: 0.4810, 정밀도 : 0.0356, 재현율:0.9125,F1 스코어:0.0685
ROC 스코어: 0.6922, PR score : 0.0343
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2418 1325]
 [  14   66]]
정확도: 0.6498, 정밀도 : 0.0474, 재현율:0.8250,F1 스코어:0.0897
ROC 스코어: 0.7355, PR score : 0.0428
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2896  847]
 [  19   61]]
정확도: 0.7735, 정밀도 : 0.0672, 재현율:0.7625,F1 스코어:0.1235
ROC 스코어: 0.7681, PR score : 0.0562
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[3212  531]
 [  24   56]]
정확도: 0.8548, 정밀도 : 0.0954, 재현율:0.7000,

In [9]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)

dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
dt_pred_proba = dt_clf.predict_proba(x_test)   

get_clf_eval(y_test, dt_pred)
get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


오차행렬
[[3271  472]
 [  33   47]]
정확도: 0.8679, 정밀도 : 0.0906, 재현율:0.5875,F1 스코어:0.1569
ROC 스코어: 0.7307, PR score : 0.0618
임곗값: 0.1
오차행렬
[[3271  472]
 [  33   47]]
정확도: 0.8679, 정밀도 : 0.0906, 재현율:0.5875,F1 스코어:0.1569
ROC 스코어: 0.7307, PR score : 0.0618
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[3271  472]
 [  33   47]]
정확도: 0.8679, 정밀도 : 0.0906, 재현율:0.5875,F1 스코어:0.1569
ROC 스코어: 0.7307, PR score : 0.0618
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[3271  472]
 [  33   47]]
정확도: 0.8679, 정밀도 : 0.0906, 재현율:0.5875,F1 스코어:0.1569
ROC 스코어: 0.7307, PR score : 0.0618
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[3271  472]
 [  33   47]]
정확도: 0.8679, 정밀도 : 0.0906, 재현율:0.5875,F1 스코어:0.1569
ROC 스코어: 0.7307, PR score : 0.0618
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[3271  472]
 [  33   47]]
정확도: 0.8679, 정밀도 : 0.0906, 재현율:0.5875,F1 스코어:0.1569
ROC 스코어: 0.7307, PR score : 0.0618
---------

In [10]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)

# key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
param_grid = {
    'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10]}
grid_search = GridSearchCV(tree
                        , param_grid = param_grid
                        , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                        , n_jobs = -1)  #-1: 모든 코어 사용

grid_search.fit(x_train, y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
dt_clf_best.fit(x_train, y_train)
dt_best_pred = dt_clf_best.predict(x_test)
dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

print("디시전트리 파라미터적용") 
get_clf_eval(y_test, dt_best_pred)
get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 6, 'min_samples_split': 4}, 최적 평균 정확도 :0.876
디시전트리 파라미터적용
오차행렬
[[3030  713]
 [  26   54]]
정확도: 0.8067, 정밀도 : 0.0704, 재현율:0.6750,F1 스코어:0.1275
ROC 스코어: 0.7423, PR score : 0.0543
임곗값: 0.1
오차행렬
[[2057 1686]
 [  14   66]]
정확도: 0.5553, 정밀도 : 0.0377, 재현율:0.8250,F1 스코어:0.0721
ROC 스코어: 0.6873, PR score : 0.0347
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2612 1131]
 [  19   61]]
정확도: 0.6992, 정밀도 : 0.0512, 재현율:0.7625,F1 스코어:0.0959
ROC 스코어: 0.7302, PR score : 0.0440
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2831  912]
 [  20   60]]
정확도: 0.7562, 정밀도 : 0.0617, 재현율:0.7500,F1 스코어:0.1141
ROC 스코어: 0.7532, PR score : 0.0515
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2961  782]
 [  25   55]]
정확도: 0.7889, 정밀도 : 0.0657, 재현율:0.6875,F1 스코어:0.1200
ROC 스코어: 0.7393, PR score : 0.0517
------------------------------------------

In [11]:
print("디시전트리") 
print(get_clf_eval(y_test, lda_pred))

디시전트리
오차행렬
[[3212  531]
 [  24   56]]
정확도: 0.8548, 정밀도 : 0.0954, 재현율:0.7000,F1 스코어:0.1679
ROC 스코어: 0.7791, PR score : 0.0731
None


In [12]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)
print("랜덤포레스트") 
get_clf_eval(y_test, rf_pred)
get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)



랜덤포레스트
오차행렬
[[3529  214]
 [  27   53]]
정확도: 0.9370, 정밀도 : 0.1985, 재현율:0.6625,F1 스코어:0.3055
ROC 스코어: 0.8027, PR score : 0.1386
임곗값: 0.1
오차행렬
[[2237 1506]
 [   7   73]]
정확도: 0.6042, 정밀도 : 0.0462, 재현율:0.9125,F1 스코어:0.0880
ROC 스코어: 0.7551, PR score : 0.0440
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2880  863]
 [  13   67]]
정확도: 0.7709, 정밀도 : 0.0720, 재현율:0.8375,F1 스코어:0.1327
ROC 스코어: 0.8035, PR score : 0.0637
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[3223  520]
 [  16   64]]
정확도: 0.8598, 정밀도 : 0.1096, 재현율:0.8000,F1 스코어:0.1928
ROC 스코어: 0.8305, PR score : 0.0919
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[3415  328]
 [  21   59]]
정확도: 0.9087, 정밀도 : 0.1525, 재현율:0.7375,F1 스코어:0.2527
ROC 스코어: 0.8249, PR score : 0.1179
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[3529  214]
 [  27   53]]
정확도: 0.9370, 정밀도 : 0.1985, 재현율:0.6625,F1 스코어:0.3055
ROC 스코어: 0.8027, PR score : 0.1386
--

In [13]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의

grid_search = {'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10],
            'n_estimators': [20,40,60,80,100]}

model = RandomForestClassifier(random_state=52)
grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                            cv = 5,verbose=0, n_jobs = -1)
grid_search.fit(x_train,y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
rf_clf_best.fit(x_train, y_train)
rf_best_pred = rf_clf_best.predict(x_test)
rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

print("랜덤포레스트 파라미터적용") 
get_clf_eval(y_test, rf_best_pred)
get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

KeyboardInterrupt: 

In [ ]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)
print("로지스틱회귀") 
get_clf_eval(y_test, lr_pred)
get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

In [ ]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV


params = {'penalty': ['l2','l1','elasticnet','None'],
        'C':[0.01,0.1,1,3,5,10]}

grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_search.fit(x_train,y_train)
print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
lr_clf_best.fit(x_train, y_train)
lr_best_pred = lr_clf_best.predict(x_test)
lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

print("로지스틱회귀 파라미터적용") 
get_clf_eval(y_test, lr_best_pred)
get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=17,
                    n_estimators=90,
                    max_depth=5,
                    learning_rate=0.03,
                    reg_lambda=2,
                    reg_alpha=0.02,
                    min_split_loss=2
                    )
xgb.fit(x_train, y_train)
xgb_pred = xgb.predict(x_test)
xgb_pred_proba = xgb.predict_proba(x_test)

print("xgboost 파라미터적용") 
get_clf_eval(y_test, xgb_pred)
get_eval_by_threshold(y_test ,xgb_pred_proba[:,1].reshape(-1,1), thresholds)